In [1]:
import pandas as pd
import numpy  as np
from config import getClient
from checkMissing import checkMissing

In [2]:
dataloc = getClient("OutFront")

In [3]:
df = pd.read_csv(dataloc+"2c_updatedHours.csv")

# Convert their date format to a standard date
df["date"] = pd.to_datetime(df["date"])

In [4]:
# Check each panel for missing dates
df = df.set_index("panel")
grp = df.groupby(level=0)

count = 0
for idx, val in grp:
    missing = checkMissing(val, "day")
    if len(missing) > 0:
        count += 1
        print("found one missing: {}".format(idx))
print("Number missing dates: {}".format(count))

Number missing dates: 0
